In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm,tnrange
from SD import stochastic_descent
from Qmodel import ground_state
import os
import pandas as pd

In [ ]:
def correlation(matrix,h):
    '''
    Given a matrix this function computes the correlation quantity Q(T) as in the paper.
    Matrix must have dimension (n_protocols,lenght_of_protocol)    
    '''
    n_row = matrix.shape[0]
    n_col = matrix.shape[1] 
    
    mean_hx = np.array([matrix[:,i].mean() for i in range(n_col)]) #mean over all protocols at fixed time
    avg_over_h = np.array([np.array([ (matrix[i,j]-mean_hx[j])**2 for i in range(n_row)]).sum()/n_row for j in range(n_col)])
    avg_over_Nt = (1/((h*h)*n_col))*(avg_over_h.sum())
    return  avg_over_Nt

In [ ]:
#SD parameters
L = 1
h = 4
h_list = [-h,h]
nsteps = 100
exp_decay = False
metropolis = False

print("-------------PARAMETERS for SD---------------")
print("Number of qubits (L):", L)
print("Magnetic fields(h):", h)
print("Timesteps (n_steps):", nsteps)
print("\n")
print("\n")


#------------PREPARE QSTART E QTARGET
qstart = ground_state(L, -2)
qtarget = ground_state(L, +2)



#-----------PREPARE PARAMETERS FOR PLOTTING
times_first_part=np.arange(0.1,1,0.1)
times_second_part=np.arange(1,4.1,0.1)
times=np.concatenate([times_first_part,times_second_part])
iter_for_each_time = 10

fidelity_for_txt = []
print("--------------PARAMETERS for Plotting-------------")
print("Timegrid:", times)
print("Repetition at each timestep:", iter_for_each_time)
print("\n")

In [ ]:
#UNCOMMENT TO RUN
print("WARNING! Attenzione a non sovrascrivere")


custom_name_dir = "test"
os.system("mkdir "+custom_name_dir)
os.system("mkdir "+custom_name_dir+"/protocols")

intermediete_result = True

for T in times:
    temp_fid = []
    best_prot = []
    print("Running time:", T)
    for iters in tnrange(iter_for_each_time):
        best_protocol, fidelity = stochastic_descent(qstart, qtarget, L, T, nsteps, 
                        field_list = h_list, verbose = True, check_norm = True)

        temp_fid.append(fidelity[-1]) 
        best_prot.append(best_protocol) 
        
    fidelity_for_txt.append(temp_fid) 
    best_prot = np.array([best_prot])
    with open(custom_name_dir +'/protocols/testT'+str(round(T, 2))+'.npy', 'wb') as f:
        np.save(f,best_prot)
    f.close()
    
    if intermediete_result and T !=0: 
        data = np.load(custom_name_dir +'/protocols/testT'+str(round(T, 3))+'.npy')[0,:,:] #first dimension is redundant '''
        print("Mean fidelity:", np.array(temp_fid).mean())
        print("Q value is:", correlation(data, h))
        print("\n")
        
#fidelity values are saved at the end
np.savetxt(custom_name_dir + '/fidelity_SD.txt', fidelity_for_txt, delimiter = ',',header="Matrix with as entries the values of fidelity dimension times x iterations")


In [ ]:
#PLOT 
#os.chdir("C:\Users\GUIDA\Desktop\CODE_MONTANGERO\SD_WITH_REPLACEMENT")
q=[]
for T in times[1:]:
    data = np.load(custom_name_dir +"/protocols/testT"+str(round(T, 2))+".npy")[0,:,:] #first dimension is redundant 
    q.append(correlation(data,h))
    
loaded_fidelity = pd.read_csv(custom_name_dir +'/fidelity_SD.txt', skiprows=1,header=None)

fig, ax = plt.subplots(figsize=(10,7))
#plot Fidelity values
ax.errorbar(times,loaded_fidelity.mean(axis=1).values, yerr=loaded_fidelity.std(axis=1).values, color="r")
ax.scatter(times,loaded_fidelity.mean(axis=1).values,color='r',label="Fidelity")
#plot Q values
ax.plot(times[1:], q, marker="o", color="goldenrod", markersize=7, label="q(T)")
ax.vlines(0.5,-0.05,1.05, color="b", linestyle="-.")
ax.vlines(2.4,-0.05,1.05, color="b", linestyle="-.")
ax.set_ylim(-0.05,1.05)
ax.set_title(r"Phase Diagram"+"\n"+"Iterations for Each Point = "+str(iter_for_each_time), fontsize=18)
ax.set_xlabel("T [a.u.]", fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.legend(fontsize=13, loc=5)
ax.grid()
fig.tight_layout()
fig.savefig("phase_diagram_L="+str(L)+".pdf")
plt.show()